# Imports

In [ ]:
from ultralytics import YOLO
import os
from scipy.io import loadmat
import numpy as np
import shutil
import math
import scipy.io

In [43]:
# load model YOLO v8
model = YOLO("yolov8m-pose.pt")

# configurations
N = 10
MPII_IMAGES_DIR = "./mpii/images"
IMAGE_FILES = [os.path.join(MPII_IMAGES_DIR, file) for file in os.listdir(MPII_IMAGES_DIR) if file.endswith(('.jpg'))][:N]
yolo_images_dir = "./YOLO_images"

count = 0
results_list = []

# USE ANNOTATION MPII
annotations_data = loadmat("./mpii/annotations/mpii_human_pose.mat")
#print("annotations_data", annotations_data)
#annolist_test = annotations_data['RELEASE'] ['annolist']


#print("Data Structure annolist test", annolist_test)
#name_images= annolist_test[0]['image']['name']
#print(name_images)
# annolist_test  tridimensional matrix
##############################################################

annolist = annotations_data['RELEASE'][0][0]['annolist'] 


for entry in annolist:
    # get first 10 images names
    image_names = entry['image'][:10]
    print(image_names)







########################################################################3

[array([[(array(['037454012.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['095071431.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['073199394.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['059865848.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')])
 array([[(array(['015601864.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['015599452.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['005808361.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['086617615.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')])
 array([[(array(['060111501.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]) array([[(array(['070807258.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')])]


In [24]:
for file in os.listdir(MPII_IMAGES_DIR):
    if file.endswith('.jpg'):
        image_file = os.path.join(MPII_IMAGES_DIR, file)
        shutil.copy(image_file, yolo_images_dir)
        
        # execute detection for YOLO_images directory
        results = model(source=image_file, show=False, conf=0.3, save=False)
        results_list.append(results)
        #print("RESULTS LIST", results_list)
        count += 1
        if count >= N:
            break


image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000111209.jpg: 480x640 1 person, 953.8ms
Speed: 5.0ms preprocess, 953.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000114595.jpg: 384x640 2 persons, 733.5ms
Speed: 0.0ms preprocess, 733.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000120361.jpg: 480x640 11 persons, 865.2ms
Speed: 0.0ms preprocess, 865.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000124344.jpg: 384x640 10 persons, 735.8ms
Speed: 0.0ms preprocess, 735.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\O

In [3]:
# acess and print predictid keypoints
for i,result in enumerate(results_list):
    for r in result:
        #print(len(result))
        image_name = r.path.split('\\')[-1]
        #print(f"image {i+1} ({image_name})")
        xyn = r.keypoints.xyn
        for j, pose in enumerate(xyn):
            #print(f"pose {j+1}")
            for k, landmark in enumerate(pose):
                x, y = landmark
                #print(f"keypoints {k+1}:\t ({x},\t {y})") 

